# Table of Contents

### Imports

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

In [2]:
import requests
from bs4 import BeautifulSoup

### Preparing Data

In [3]:
# Static strings we are going to use
baseUrl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_b_list=1&ww_i_reportmodel=133685247'
ampersand = '&'

In [7]:
# Get a response object from the base url
r = requests.get(baseUrl, timeout=30)

# Transfer the response object into a BeautifulSoup object
soup = BeautifulSoup(r.text, 'lxml')

In [9]:
# There is enormous content in the file, but we can see that all the filters are described in the 'form' label.
print(soup.form.prettify())

<form action="!GEDPUBLICREPORTS.filter" method="GET" name="f">
 <input name="ww_b_list" type="hidden" value="1"/>
 <input name="ww_i_reportmodel" type="hidden" value="133685247"/>
 <input name="ww_c_langue" type="hidden" value=""/>
 <h1 id="titre">
  Liste des étudiants inscrits par semestre
 </h1>
 <table border="0" id="format">
  <tr>
   <th>
    Format:
   </th>
  </tr>
  <tr>
   <td>
    <input checked="" name="ww_i_reportModelXsl" type="radio" value="133685270"/>
    html
   </td>
  </tr>
  <tr>
   <td>
    <input name="ww_i_reportModelXsl" type="radio" value="133685271"/>
    xls
   </td>
  </tr>
 </table>
 <h1>
 </h1>
 <table border="0" id="filtre">
  <tr>
   <th>
    Unité académique
   </th>
   <td>
    <input name="zz_x_UNITE_ACAD" type="hidden" value=""/>
    <select name="ww_x_UNITE_ACAD" onchange="document.f.zz_x_UNITE_ACAD.value=document.f.ww_x_UNITE_ACAD.options[document.f.ww_x_UNITE_ACAD.selectedIndex].text">
     <option value="null">
     </option>
     <option value=

In [15]:
# each 'option' label represents an option in a certain filter. List all to find the options we want to use.
soup.form.find_all('option')

[<option value="null"></option>,
 <option value="942293">Architecture</option>,
 <option value="246696">Chimie et génie chimique</option>,
 <option value="943282">Cours de mathématiques spéciales</option>,
 <option value="637841336">EME (EPFL Middle East)</option>,
 <option value="942623">Génie civil</option>,
 <option value="944263">Génie mécanique</option>,
 <option value="943936">Génie électrique et électronique </option>,
 <option value="2054839157">Humanités digitales</option>,
 <option value="249847">Informatique</option>,
 <option value="120623110">Ingénierie financière</option>,
 <option value="946882">Management de la technologie</option>,
 <option value="944590">Mathématiques</option>,
 <option value="945244">Microtechnique</option>,
 <option value="945571">Physique</option>,
 <option value="944917">Science et génie des matériaux</option>,
 <option value="942953">Sciences et ingénierie de l'environnement</option>,
 <option value="945901">Sciences et technologies du vivant</op

In [12]:
# Each 'input' label represents a certain filter. List all of them to find out their names,
# so we can concatenate our URLs to do queries.
soup.form.find_all('input')

[<input name="ww_b_list" type="hidden" value="1"/>,
 <input name="ww_i_reportmodel" type="hidden" value="133685247"/>,
 <input name="ww_c_langue" type="hidden" value=""/>,
 <input checked="" name="ww_i_reportModelXsl" type="radio" value="133685270"/>,
 <input name="ww_i_reportModelXsl" type="radio" value="133685271"/>,
 <input name="zz_x_UNITE_ACAD" type="hidden" value=""/>,
 <input name="zz_x_PERIODE_ACAD" type="hidden" value=""/>,
 <input name="zz_x_PERIODE_PEDAGO" type="hidden" value=""/>,
 <input name="zz_x_HIVERETE" type="hidden" value=""/>,
 <input name="dummy" type="submit" value="ok"/>]

In [ ]:
# Before we start queries, we note down first the values and names we need to use